This is the initial notebook to explore the data and run the first models. 

In [1]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder

from fastai.imports import *
from fastai.tabular.all import *

from numpy import random
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_log_error
from sklearn import metrics
from sklearn.model_selection import KFold, cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from scipy.stats import skew, norm
from catboost import CatBoostRegressor
from catboost import Pool

Import additional models

In [2]:
import shap
from sklearn.svm import SVR
from lightgbm import LGBMRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeRegressor
from mlxtend.regressor import StackingRegressor
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import VotingRegressor

In [3]:
test = pd.read_csv('test.csv')
sample = pd.read_csv('sample_submission.csv')
train = pd.read_csv('train.csv')

Based on initial analysis the train and test datasets have similar characteristics, so it will be easier to combine them for imputation and data analysis work. 

In [4]:
train_test = pd.concat([train, test], ignore_index=True)

In [5]:
pd.isnull(train_test).sum()[pd.isnull(train_test).sum() > 0]

MSZoning           4
LotFrontage      486
Alley           2721
Utilities          2
Exterior1st        1
Exterior2nd        1
MasVnrType        24
MasVnrArea        23
BsmtQual          81
BsmtCond          82
BsmtExposure      82
BsmtFinType1      79
BsmtFinSF1         1
BsmtFinType2      80
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
Electrical         1
BsmtFullBath       2
BsmtHalfBath       2
KitchenQual        1
Functional         2
FireplaceQu     1420
GarageType       157
GarageYrBlt      159
GarageFinish     159
GarageCars         1
GarageArea         1
GarageQual       159
GarageCond       159
PoolQC          2909
Fence           2348
MiscFeature     2814
SaleType           1
SalePrice       1459
dtype: int64

It looks like Alley, FireplaceQu, PoolQC, Fence and MiscFeature have significant numbers of missing data. So it will be best to eliminate those rows. There are a number of rows that have less than 5 rows with missing data. Since some of these are categorical and some are continuous data, their missing data will be replaced with the most frequent value. 

In [6]:
drop_high_nan=['Alley','FireplaceQu','PoolQC','Fence','MiscFeature']
train_test=train_test.drop(drop_high_nan,axis=1)
small_nan_cols = ['MSZoning', 'Utilities', 'Exterior1st', 'Exterior2nd', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 
                  'TotalBsmtSF', 'Electrical', 'BsmtFullBath', 'BsmtHalfBath', 'KitchenQual', 'Functional', 'GarageCars', 
                  'GarageArea','SaleType', 'SaleCondition']
small_impute = SimpleImputer(strategy='most_frequent')
train_test[small_nan_cols] = pd.DataFrame(small_impute.fit_transform(train_test[small_nan_cols]),columns=small_nan_cols)

The following columns seem to have one value significantly larger than the rest, and it would probably be best to use the mode, or most common, value to feel each NaN value: MasVnrType, MasVnrArea, BsmtCond, BsmtExposure, BsmtFinType2, GarageType, GarageFinish, GarageQual, and GarageCond. That is represents 9 out of the 13 columns. 

BsmtQual has two values larger than the rest: Gd and TA. But it only has 2.8% NaNs, so simply using the mode might be good enough.  

GarageYrBlt has 59 NaNs out 2919 rows which is only 2%. It has a dispersed set of values, so it might be easiest just to have any NaNs have the same value as YearBuilt. 

BsmtFinType1 has only 2.7% value of NaNs, and most two of its largest values are GLQ and Unf. It might be easiest to use the mode here. 

LotFrontage has 486 NaNs out of 2919 rows which is a pretty high 16.7%. It has a dispersed range of values, but looking at its characteristics from the describe function above, it seems to have a pretty even distribution with a mean of 10,168 and a median of 9,453. So using the mean to fill in the NaNs seems reasonable. If it turns out this value has a high impact on the predictability of the SalePrice, then it might be good to revisit this assumption. 

In [7]:
mode_cols = ['MasVnrType', 'MasVnrArea', 'BsmtCond', 'BsmtExposure', 'BsmtFinType2', 'GarageType', 'GarageFinish', 
             'GarageQual','GarageCond', 'BsmtQual', 'BsmtFinType1']
mode_impute = SimpleImputer(strategy='most_frequent')
train_test[mode_cols] = pd.DataFrame(mode_impute.fit_transform(train_test[mode_cols]),columns=mode_cols)
train_test['LotFrontage'].fillna((train_test['LotFrontage'].mean()), inplace=True)
train_test['GarageYrBlt'] = train_test['GarageYrBlt'].fillna(train_test['YearBuilt'])

In [8]:
pd.isnull(train_test).sum()[pd.isnull(train_test).sum() > 0]

SalePrice    1459
dtype: int64

In [9]:
train_test['BsmtQual'].unique()

array(['Gd', 'TA', 'Ex', 'Fa'], dtype=object)

In [10]:
train_test['BsmtFinType1'].unique()

array(['GLQ', 'ALQ', 'Unf', 'Rec', 'BLQ', 'LwQ'], dtype=object)

In [11]:
train_test.BsmtQual = train_test.BsmtQual.replace({"Ex": 110, "Gd": 95, "TA": 85, "Fa": 75, "Po": 60, "NA": 0})

In [12]:
train_test.BsmtFinType1 = train_test.BsmtFinType1.replace({"GLQ": 6, "ALQ": 5, "BLQ": 4, "Rec": 3, "LwQ": 2, "Unf": 1,
                                                         "NA": 0})

It will be necessary to identify all the columns that have non-numeric object values and then convert them to numeric values. 

In [13]:
obj_cols = list(train_test.select_dtypes(['object']).columns)
obj_cols

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'MasVnrArea',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinSF1',
 'BsmtFinType2',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'BsmtFullBath',
 'BsmtHalfBath',
 'KitchenQual',
 'Functional',
 'GarageType',
 'GarageFinish',
 'GarageCars',
 'GarageArea',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [14]:
for column in obj_cols:
     train_test[column] = pd.factorize(train_test[column], sort=True)[0]

In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)


To create some new columns that might compound the effects of the existing columns with higher impact on the xgb score. 

In [15]:
train_test['QualCondSum'] = train_test['OverallQual'] + train_test['OverallCond']
train_test['RemodTime'] = train_test['YearRemodAdd'] - train_test['YearBuilt']
train_test['BsmtFinTypeSF1'] = train_test['BsmtFinType1'] * train_test['BsmtFinSF1']
train_test['TotalFlrSF'] = train_test['1stFlrSF'] + train_test['2ndFlrSF']
train_test['TotalFinSF'] = train_test['GrLivArea'] + train_test['BsmtFinSF1']
train_test['GarageCarArea'] = train_test['GarageArea'] * train_test['GarageCars']
train_test['TotalSF'] = train_test['1stFlrSF'] + train_test['2ndFlrSF'] + train_test['TotalBsmtSF']

To now eliminate the columns that have negative or zero influence on the xgb score. 

In [16]:
train_test.drop(['Utilities','Street','TotRmsAbvGrd','3SsnPorch','YrSold','Exterior2nd','LotConfig',
                              'HouseStyle','EnclosedPorch','WoodDeckSF','Foundation','RoofMatl','Electrical',
                              'GarageType','LotFrontage','SaleType','MoSold','BsmtExposure','1stFlrSF',
                              'BsmtFinSF1','Exterior1st','KitchenQual','TotalFlrSF'], axis=1, inplace=True)

To look at how many features have a skew above 0.5, since high skew can be an issue in regression analysis. 

In [17]:
skew_features = train_test.apply(lambda x: skew(x)).sort_values(ascending=False)

In [18]:
high_skew = skew_features[skew_features > 0.55]
skew_index = high_skew.index

In [19]:
skew_index

Index(['MiscVal', 'PoolArea', 'LotArea', 'LowQualFinSF', 'Heating',
       'Condition2', 'LandSlope', 'KitchenAbvGr', 'ScreenPorch',
       'BsmtHalfBath', 'BsmtFinSF2', 'Condition1', 'OpenPorchSF', 'BldgType',
       'RemodTime', 'MasVnrArea', 'RoofStyle', 'MSSubClass', 'GarageCarArea',
       'GrLivArea', 'TotalFinSF', 'BsmtFinTypeSF1', 'TotalSF', '2ndFlrSF',
       'BsmtQual', 'Fireplaces', 'HalfBath', 'BsmtFullBath', 'OverallCond'],
      dtype='object')

In [20]:
print("There are {} numerical features with Skew > 0.55 :".format(high_skew.shape[0]))
skewness = pd.DataFrame({'Skew' :high_skew})
skew_features.head(30)

There are 29 numerical features with Skew > 0.55 :


MiscVal           21.947195
PoolArea          16.898328
LotArea           12.822431
LowQualFinSF      12.088761
Heating           12.078788
Condition2        12.060093
LandSlope          4.975157
KitchenAbvGr       4.302254
ScreenPorch        3.946694
BsmtHalfBath       3.931594
BsmtFinSF2         3.476562
Condition1         2.983114
OpenPorchSF        2.535114
BldgType           2.192261
RemodTime          2.063712
MasVnrArea         1.556310
RoofStyle          1.553307
MSSubClass         1.375457
GarageCarArea      1.296672
GrLivArea          1.269358
TotalFinSF         1.159630
BsmtFinTypeSF1     0.932490
TotalSF            0.922771
2ndFlrSF           0.861675
BsmtQual           0.796302
Fireplaces         0.733495
HalfBath           0.694566
BsmtFullBath       0.624832
OverallCond        0.570312
HeatingQC          0.486656
dtype: float64

To normalize the features with skew above 0.5.

In [21]:
for i in skew_index:
    train_test[i] = np.log1p(train_test[i])

divide by zero encountered in log1p


To create a column with the log of the SalePrice to match the evaluation process in the contest. 

In [22]:
train_test['LogSalePrice'] = train_test['SalePrice'].apply(np.log)

To separate the train_test dataset back into the train and test datasets, identify the independent and dependent columns, and create the validation split.

In [23]:
train = train_test[train_test['SalePrice'].notnull()].copy()
test = train_test[train_test['SalePrice'].isnull()].drop(['SalePrice','LogSalePrice'],axis=1)
X = train.drop(['SalePrice','LogSalePrice'],axis=1)
y = train.LogSalePrice

In [24]:
X,y = shuffle(X,y, random_state=42)
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)

To define the models

In [25]:
cat_model = CatBoostRegressor()
xgb_model = xgb.XGBRegressor()
lr_model = LinearRegression()
br_model = BayesianRidge(compute_score=True)
lightgbm_model = LGBMRegressor(objective='regression')
svr_model = SVR()
dtr_model = DecisionTreeRegressor()
rfr_model = RandomForestRegressor()
gbr_model = GradientBoostingRegressor()

To define stacking models.

In [27]:
stack_gen = StackingRegressor(regressors=(CatBoostRegressor(),LinearRegression(),BayesianRidge(),
                                          GradientBoostingRegressor(),RandomForestRegressor()),
                              meta_regressor = CatBoostRegressor(),
                              use_features_in_secondary = True)

vote_gen = VotingRegressor(estimators=[('Cat_Boost_Reg', CatBoostRegressor()),('Linear_Reg', LinearRegression()),
                                       ('Bayesian_Ridge_Reg', BayesianRidge()),
                                       ('Grad_Boost_Reg', GradientBoostingRegressor()),
                                       ('LGBM_Reg', LGBMRegressor(objective='regression')),
                                       ('Random_Forest_Reg', RandomForestRegressor())
                                            ])

In [26]:
model_names = ['CatBoostRegressor','XGBRegressor','LinearRegression','BayesianRidge','LGBMRegressor',
               'SVR','DecisionTreeRegressor','RandomForestRegressor','GradientBoostingRegressor',
               'StackingRegressor','VotingRegressor']

In [28]:
models = [cat_model,xgb_model,lr_model,br_model,lightgbm_model,svr_model,dtr_model,rfr_model,gbr_model,
          stack_gen,vote_gen]

To generate the scores of all the models.

In [29]:
score_rmse = []
for i in models:
    rmse = np.sqrt(-cross_val_score(i, X, y, scoring="neg_mean_squared_error", cv=6))
    ave_rmse = np.mean(rmse)
    score_rmse.append(ave_rmse)

Learning rate set to 0.042228
0:	learn: 0.3791147	total: 144ms	remaining: 2m 23s
1:	learn: 0.3683561	total: 148ms	remaining: 1m 13s
2:	learn: 0.3582837	total: 152ms	remaining: 50.5s
3:	learn: 0.3486877	total: 155ms	remaining: 38.7s
4:	learn: 0.3394573	total: 159ms	remaining: 31.6s
5:	learn: 0.3308171	total: 161ms	remaining: 26.8s
6:	learn: 0.3219869	total: 164ms	remaining: 23.3s
7:	learn: 0.3142303	total: 167ms	remaining: 20.7s
8:	learn: 0.3065404	total: 170ms	remaining: 18.7s
9:	learn: 0.2989892	total: 172ms	remaining: 17s
10:	learn: 0.2917168	total: 175ms	remaining: 15.7s
11:	learn: 0.2847115	total: 177ms	remaining: 14.6s
12:	learn: 0.2783108	total: 180ms	remaining: 13.6s
13:	learn: 0.2721388	total: 182ms	remaining: 12.8s
14:	learn: 0.2655222	total: 185ms	remaining: 12.1s
15:	learn: 0.2594223	total: 187ms	remaining: 11.5s
16:	learn: 0.2531697	total: 189ms	remaining: 10.9s
17:	learn: 0.2483696	total: 192ms	remaining: 10.4s
18:	learn: 0.2429591	total: 194ms	remaining: 10s
19:	learn: 0.

169:	learn: 0.0952475	total: 585ms	remaining: 2.85s
170:	learn: 0.0949968	total: 588ms	remaining: 2.85s
171:	learn: 0.0948288	total: 591ms	remaining: 2.85s
172:	learn: 0.0946576	total: 595ms	remaining: 2.84s
173:	learn: 0.0945805	total: 599ms	remaining: 2.84s
174:	learn: 0.0944148	total: 603ms	remaining: 2.84s
175:	learn: 0.0943572	total: 606ms	remaining: 2.84s
176:	learn: 0.0942503	total: 609ms	remaining: 2.83s
177:	learn: 0.0940732	total: 614ms	remaining: 2.83s
178:	learn: 0.0939563	total: 620ms	remaining: 2.85s
179:	learn: 0.0938421	total: 624ms	remaining: 2.84s
180:	learn: 0.0935758	total: 628ms	remaining: 2.84s
181:	learn: 0.0934292	total: 633ms	remaining: 2.85s
182:	learn: 0.0933822	total: 637ms	remaining: 2.84s
183:	learn: 0.0931040	total: 641ms	remaining: 2.84s
184:	learn: 0.0929246	total: 650ms	remaining: 2.86s
185:	learn: 0.0928655	total: 654ms	remaining: 2.86s
186:	learn: 0.0927237	total: 660ms	remaining: 2.87s
187:	learn: 0.0926314	total: 664ms	remaining: 2.87s
188:	learn: 

378:	learn: 0.0703587	total: 1.17s	remaining: 1.91s
379:	learn: 0.0702188	total: 1.17s	remaining: 1.91s
380:	learn: 0.0701265	total: 1.17s	remaining: 1.9s
381:	learn: 0.0700605	total: 1.17s	remaining: 1.9s
382:	learn: 0.0699580	total: 1.18s	remaining: 1.9s
383:	learn: 0.0698740	total: 1.18s	remaining: 1.89s
384:	learn: 0.0697960	total: 1.18s	remaining: 1.89s
385:	learn: 0.0696958	total: 1.19s	remaining: 1.89s
386:	learn: 0.0696773	total: 1.19s	remaining: 1.88s
387:	learn: 0.0695780	total: 1.19s	remaining: 1.88s
388:	learn: 0.0695295	total: 1.19s	remaining: 1.87s
389:	learn: 0.0693884	total: 1.2s	remaining: 1.87s
390:	learn: 0.0692783	total: 1.2s	remaining: 1.86s
391:	learn: 0.0691555	total: 1.2s	remaining: 1.86s
392:	learn: 0.0690823	total: 1.2s	remaining: 1.86s
393:	learn: 0.0689472	total: 1.2s	remaining: 1.85s
394:	learn: 0.0688038	total: 1.21s	remaining: 1.85s
395:	learn: 0.0687219	total: 1.21s	remaining: 1.84s
396:	learn: 0.0686879	total: 1.21s	remaining: 1.84s
397:	learn: 0.068616

611:	learn: 0.0527599	total: 1.75s	remaining: 1.11s
612:	learn: 0.0527140	total: 1.75s	remaining: 1.1s
613:	learn: 0.0526390	total: 1.75s	remaining: 1.1s
614:	learn: 0.0525383	total: 1.76s	remaining: 1.1s
615:	learn: 0.0524891	total: 1.76s	remaining: 1.1s
616:	learn: 0.0524352	total: 1.76s	remaining: 1.09s
617:	learn: 0.0523974	total: 1.76s	remaining: 1.09s
618:	learn: 0.0523613	total: 1.77s	remaining: 1.09s
619:	learn: 0.0522735	total: 1.77s	remaining: 1.08s
620:	learn: 0.0522688	total: 1.77s	remaining: 1.08s
621:	learn: 0.0521876	total: 1.77s	remaining: 1.08s
622:	learn: 0.0521072	total: 1.78s	remaining: 1.07s
623:	learn: 0.0520384	total: 1.78s	remaining: 1.07s
624:	learn: 0.0519356	total: 1.78s	remaining: 1.07s
625:	learn: 0.0518761	total: 1.78s	remaining: 1.07s
626:	learn: 0.0518720	total: 1.79s	remaining: 1.06s
627:	learn: 0.0518283	total: 1.79s	remaining: 1.06s
628:	learn: 0.0517146	total: 1.79s	remaining: 1.06s
629:	learn: 0.0516977	total: 1.79s	remaining: 1.05s
630:	learn: 0.05

818:	learn: 0.0427212	total: 2.34s	remaining: 517ms
819:	learn: 0.0426637	total: 2.34s	remaining: 514ms
820:	learn: 0.0425827	total: 2.35s	remaining: 512ms
821:	learn: 0.0425014	total: 2.35s	remaining: 509ms
822:	learn: 0.0424734	total: 2.35s	remaining: 507ms
823:	learn: 0.0424702	total: 2.36s	remaining: 504ms
824:	learn: 0.0424155	total: 2.37s	remaining: 502ms
825:	learn: 0.0424131	total: 2.37s	remaining: 500ms
826:	learn: 0.0423683	total: 2.38s	remaining: 498ms
827:	learn: 0.0422956	total: 2.38s	remaining: 495ms
828:	learn: 0.0422403	total: 2.39s	remaining: 493ms
829:	learn: 0.0421777	total: 2.39s	remaining: 490ms
830:	learn: 0.0421517	total: 2.4s	remaining: 488ms
831:	learn: 0.0421260	total: 2.4s	remaining: 486ms
832:	learn: 0.0420709	total: 2.41s	remaining: 483ms
833:	learn: 0.0419927	total: 2.42s	remaining: 481ms
834:	learn: 0.0419631	total: 2.42s	remaining: 478ms
835:	learn: 0.0419059	total: 2.42s	remaining: 476ms
836:	learn: 0.0418609	total: 2.43s	remaining: 473ms
837:	learn: 0.

...

257:	learn: 0.0840508	total: 1.2s	remaining: 3.46s
258:	learn: 0.0840031	total: 1.21s	remaining: 3.46s
259:	learn: 0.0838185	total: 1.21s	remaining: 3.46s
260:	learn: 0.0836956	total: 1.22s	remaining: 3.45s
261:	learn: 0.0835662	total: 1.23s	remaining: 3.45s
262:	learn: 0.0834791	total: 1.23s	remaining: 3.45s
263:	learn: 0.0834384	total: 1.24s	remaining: 3.46s
264:	learn: 0.0833106	total: 1.25s	remaining: 3.45s
265:	learn: 0.0831810	total: 1.25s	remaining: 3.45s
266:	learn: 0.0830687	total: 1.25s	remaining: 3.45s
267:	learn: 0.0828751	total: 1.26s	remaining: 3.44s
268:	learn: 0.0828176	total: 1.27s	remaining: 3.44s
269:	learn: 0.0827340	total: 1.27s	remaining: 3.44s
270:	learn: 0.0826174	total: 1.28s	remaining: 3.43s
271:	learn: 0.0825824	total: 1.28s	remaining: 3.43s
272:	learn: 0.0825457	total: 1.28s	remaining: 3.42s
273:	learn: 0.0824172	total: 1.29s	remaining: 3.42s
274:	learn: 0.0822952	total: 1.29s	remaining: 3.41s
275:	learn: 0.0821802	total: 1.3s	remaining: 3.4s
276:	learn: 0.0

438:	learn: 0.0665229	total: 2s	remaining: 2.56s
439:	learn: 0.0663739	total: 2.01s	remaining: 2.56s
440:	learn: 0.0662287	total: 2.01s	remaining: 2.55s
441:	learn: 0.0660943	total: 2.02s	remaining: 2.55s
442:	learn: 0.0659817	total: 2.02s	remaining: 2.54s
443:	learn: 0.0659102	total: 2.02s	remaining: 2.54s
444:	learn: 0.0658956	total: 2.03s	remaining: 2.53s
445:	learn: 0.0658346	total: 2.03s	remaining: 2.53s
446:	learn: 0.0657364	total: 2.04s	remaining: 2.52s
447:	learn: 0.0656659	total: 2.04s	remaining: 2.52s
448:	learn: 0.0655854	total: 2.05s	remaining: 2.51s
449:	learn: 0.0654150	total: 2.05s	remaining: 2.51s
450:	learn: 0.0653273	total: 2.06s	remaining: 2.5s
451:	learn: 0.0652513	total: 2.06s	remaining: 2.5s
452:	learn: 0.0651569	total: 2.06s	remaining: 2.49s
453:	learn: 0.0650872	total: 2.07s	remaining: 2.49s
454:	learn: 0.0650285	total: 2.07s	remaining: 2.48s
455:	learn: 0.0649707	total: 2.08s	remaining: 2.48s
456:	learn: 0.0649250	total: 2.08s	remaining: 2.47s
457:	learn: 0.064

627:	learn: 0.0528368	total: 2.82s	remaining: 1.67s
628:	learn: 0.0527593	total: 2.82s	remaining: 1.66s
629:	learn: 0.0527177	total: 2.83s	remaining: 1.66s
630:	learn: 0.0526345	total: 2.83s	remaining: 1.66s
631:	learn: 0.0525902	total: 2.83s	remaining: 1.65s
632:	learn: 0.0525030	total: 2.84s	remaining: 1.65s
633:	learn: 0.0524483	total: 2.84s	remaining: 1.64s
634:	learn: 0.0523888	total: 2.85s	remaining: 1.64s
635:	learn: 0.0523844	total: 2.85s	remaining: 1.63s
636:	learn: 0.0523233	total: 2.86s	remaining: 1.63s
637:	learn: 0.0522494	total: 2.86s	remaining: 1.62s
638:	learn: 0.0521977	total: 2.87s	remaining: 1.62s
639:	learn: 0.0521401	total: 2.87s	remaining: 1.61s
640:	learn: 0.0520829	total: 2.87s	remaining: 1.61s
641:	learn: 0.0520195	total: 2.88s	remaining: 1.6s
642:	learn: 0.0519586	total: 2.88s	remaining: 1.6s
643:	learn: 0.0519091	total: 2.89s	remaining: 1.59s
644:	learn: 0.0518616	total: 2.89s	remaining: 1.59s
645:	learn: 0.0518010	total: 2.89s	remaining: 1.59s
646:	learn: 0.

806:	learn: 0.0430972	total: 3.58s	remaining: 857ms
807:	learn: 0.0430445	total: 3.59s	remaining: 853ms
808:	learn: 0.0430243	total: 3.59s	remaining: 848ms
809:	learn: 0.0429775	total: 3.6s	remaining: 844ms
810:	learn: 0.0429176	total: 3.6s	remaining: 839ms
811:	learn: 0.0429141	total: 3.6s	remaining: 835ms
812:	learn: 0.0428367	total: 3.61s	remaining: 830ms
813:	learn: 0.0427771	total: 3.61s	remaining: 826ms
814:	learn: 0.0427199	total: 3.62s	remaining: 822ms
815:	learn: 0.0426689	total: 3.62s	remaining: 817ms
816:	learn: 0.0426180	total: 3.63s	remaining: 813ms
817:	learn: 0.0426148	total: 3.63s	remaining: 808ms
818:	learn: 0.0425602	total: 3.63s	remaining: 803ms
819:	learn: 0.0425006	total: 3.64s	remaining: 799ms
820:	learn: 0.0424656	total: 3.64s	remaining: 794ms
821:	learn: 0.0424072	total: 3.65s	remaining: 790ms
822:	learn: 0.0423815	total: 3.65s	remaining: 785ms
823:	learn: 0.0423335	total: 3.65s	remaining: 781ms
824:	learn: 0.0422822	total: 3.66s	remaining: 776ms
825:	learn: 0.0

977:	learn: 0.0361352	total: 4.37s	remaining: 98.3ms
978:	learn: 0.0361104	total: 4.38s	remaining: 93.9ms
979:	learn: 0.0360688	total: 4.38s	remaining: 89.5ms
980:	learn: 0.0360099	total: 4.39s	remaining: 85ms
981:	learn: 0.0359594	total: 4.4s	remaining: 80.6ms
982:	learn: 0.0359145	total: 4.4s	remaining: 76.2ms
983:	learn: 0.0358684	total: 4.41s	remaining: 71.7ms
984:	learn: 0.0358254	total: 4.42s	remaining: 67.2ms
985:	learn: 0.0358216	total: 4.42s	remaining: 62.8ms
986:	learn: 0.0357599	total: 4.43s	remaining: 58.3ms
987:	learn: 0.0356999	total: 4.43s	remaining: 53.9ms
988:	learn: 0.0356667	total: 4.44s	remaining: 49.4ms
989:	learn: 0.0356251	total: 4.45s	remaining: 44.9ms
990:	learn: 0.0355858	total: 4.45s	remaining: 40.4ms
991:	learn: 0.0355660	total: 4.46s	remaining: 36ms
992:	learn: 0.0355318	total: 4.46s	remaining: 31.5ms
993:	learn: 0.0354898	total: 4.47s	remaining: 27ms
994:	learn: 0.0354502	total: 4.47s	remaining: 22.5ms
995:	learn: 0.0353933	total: 4.48s	remaining: 18ms
996

In [30]:
score_rmse

[0.12095301844965532,
 0.13790232175471712,
 0.1362612920119789,
 0.13589835434767175,
 0.1321053802514074,
 0.24951637229057255,
 0.20907513416030135,
 0.13767640140140183,
 0.12713855396027465,
 0.12341549266780161,
 0.12377551530042086]

In [31]:
model_scores = list(zip(model_names, score_rmse))

In [32]:
model_scores

[('CatBoostRegressor', 0.12095301844965532),
 ('XGBRegressor', 0.13790232175471712),
 ('LinearRegression', 0.1362612920119789),
 ('BayesianRidge', 0.13589835434767175),
 ('LGBMRegressor', 0.1321053802514074),
 ('SVR', 0.24951637229057255),
 ('DecisionTreeRegressor', 0.20907513416030135),
 ('RandomForestRegressor', 0.13767640140140183),
 ('GradientBoostingRegressor', 0.12713855396027465),
 ('StackingRegressor', 0.12341549266780161),
 ('VotingRegressor', 0.12377551530042086)]